In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from deeplift_shap import DeepLiftShap

from diveslowlearnfast.train import helper as train_helper
from diveslowlearnfast.models import SlowFast, load_checkpoint
from diveslowlearnfast.config import Config

import logging

In [6]:
logging.basicConfig(level=logging.DEBUG)

cfg = Config()
cfg.DATA.DATASET_PATH = '/Users/youritomassen/Projects/xai/data/Diving48/'
cfg.TRAIN.BATCH_SIZE = 1
cfg.DATA.NUM_FRAMES = 16
device = torch.device('cpu')
model = SlowFast(cfg)
_, optimiser, *_ = train_helper.get_train_objects(cfg, model)
model, *_ = load_checkpoint(model, optimiser, '../misc/checkpoint.pth', device)
dl = DeepLiftShap(model)

test_loader = train_helper.get_test_objects(cfg)

In [3]:
x, y, *_ = next(iter(test_loader))

x.shape, y.shape

(torch.Size([1, 3, 16, 256, 256]), torch.Size([1]))

In [4]:
x_fast = x[:]
# reduce the number of frames by the alpha ratio
# B x C x T / alpha x H x W
B, C, T, H, W = x.shape
x_slow = x[:, :, ::cfg.SLOWFAST.ALPHA]
input = [x_slow, x_fast]

In [ ]:
# generate 100 baseline samples with black pixels
baseline_values = torch.zeros((100, *x.shape[1:]))
baseline_dist_fast = baseline_values[:]
baseline_dist_slow = baseline_values[:, :, ::cfg.SLOWFAST.ALPHA]
baselines = [baseline_dist_slow, baseline_dist_fast]
attributions_slow, attributions_fast = dl.attribute(input, baselines, y, n_samples=20)

attributions_slow.shape, attributions_fast.shape

DEBUG:deeplift_shap:Sampling 20 background samples
DEBUG:deeplift_shap:Scaling input tensors and background samples using DeepLIFT rescale rule
DEBUG:deeplift_shap:Computing DeepLIFT attributions
DEBUG:deeplift_shap:Computing model outputs, input type = <class 'list'>
